### Evaluate causal maps with reanalysis data

#### Functions and imports

In [1]:
### Imports
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib as mpl
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import matplotlib.patches as patches
import cartopy.feature
import statsmodels.api as sm
from scipy import signal
from scipy.signal import detrend
import random

In [6]:
### Functions and class

class JetAnalysis:
    def __init__(self, jet_data, covariates_dict, lon1=140, lon2=295):
        """
        Initialize the class with the jet data and a dictionary of covariates.
        
        Parameters:
        - jet_data: xarray.DataArray
            The jet data with dimensions lat, lon, and time.
        - covariates_dict: dict
            Dictionary of covariates, where each key is a predictor name and each value is an xarray.DataArray.
        - lon1: int
            The starting longitude for the analysis (default is 140).
        - lon2: int
            The ending longitude for the analysis (default is 295).
        """
        self.jet_data = jet_data
        self.covariates_dict = covariates_dict
        self.lon1 = lon1
        self.lon2 = lon2
        self.jet_lat = None
        self.jet_strength = None
        self.regression_results = None
    
    def jet_lat_strength(self):
        """
        Calculate the jet latitude and jet strength from the input jet data.
        
        Returns:
        - jet_lat: np.array
            Array of calculated jet latitudes over time.
        - jet_strength: np.array
            Array of calculated jet strengths over time.
        """
        jet_30_70 = self.jet_data.sel(lat=slice(-70, -30)).sel(lon=slice(self.lon1, self.lon2)).mean(dim='lon')
        lat = jet_30_70.lat
        jet_lat = (jet_30_70 * lat).sum(dim='lat') / jet_30_70.sum(dim='lat')
        
        strength = []
        for t, max_lat in zip(self.jet_data.time, jet_lat):
            strength.append(self.jet_data.sel(time=t).sel(lat=max_lat, method='nearest').sel(lon=slice(self.lon1, self.lon2)).mean(dim='lon'))
        jet_strength = np.array(strength)
        
        # Store the results as class attributes
        self.jet_lat = np.array(jet_lat.values)
        self.jet_strength = jet_strength
        
        return self.jet_lat, self.jet_strength
    
    def standardize_data(self, da):
        """
        Standardize an xarray DataArray by subtracting the mean and dividing by the standard deviation.
        
        Parameters:
        - da: xarray.DataArray
            Input data to standardize.
        
        Returns:
        - standardized_da: xarray.DataArray
            Standardized data.
        """
        mean = da.mean(dim='time')
        std_dev = da.std(dim='time')
        standardized_da = (da - mean) / std_dev
        return standardized_da
    
    def multiple_linear_regression(self, target, predictors_dict):
        """
        Perform multiple linear regression on a target time series using a dictionary of predictors.
        
        Parameters:
        - target: xarray.DataArray
            The target time series to predict.
        - predictors_dict: dict
            A dictionary where keys are predictor names and values are xarray.DataArray objects representing predictor time series.
        
        Returns:
        - results: statsmodels.regression.linear_model.RegressionResultsWrapper
            The results of the regression, including coefficients, p-values, etc.
        """
        # Convert the target and predictors to a pandas DataFrame
        df = pd.DataFrame({name: da.to_series() for name, da in predictors_dict.items()})
        
        # Ensure the target time series is aligned with predictors
        df['target'] = target.to_series()
        
        # Drop any rows with NaN values
        df.dropna(inplace=True)

        # Separate the predictors and the target
        X = df.drop(columns='target')
        y = df['target']

        # Add a constant (intercept) to the predictors
        X = sm.add_constant(X)

        # Perform the OLS regression
        model = sm.OLS(y, X)
        results = model.fit()

        return results

    def analyze(self):
        """
        Perform the full analysis: calculate metrics, standardize data, perform regression, and save results.
        
        Returns:
        - results_dict: dict
            A dictionary containing the regression coefficients, p-values, and summary.
        """
        # Step 1: Calculate the jet latitude and strength
        jet_lat, jet_strength = self.jet_lat_strength()

        # Step 2: Standardize the data
        standardized_jet_lat = self.standardize_data(xr.DataArray(jet_lat, dims=['time'], coords={'time': self.jet_data.time}))
        standardized_jet_strength = self.standardize_data(xr.DataArray(jet_strength, dims=['time'], coords={'time': self.jet_data.time}))

        # Standardize the predictors
        standardized_predictors = {name: self.standardize_data(da) for name, da in self.covariates_dict.items()}

        # Step 3: Perform multiple linear regression on both metrics
        lat_results = self.multiple_linear_regression(standardized_jet_lat, standardized_predictors)
        strength_results = self.multiple_linear_regression(standardized_jet_strength, standardized_predictors)

        # Save the regression results
        self.regression_results = {
            'jet_lat_regression': lat_results,
            'jet_strength_regression': strength_results
        }

        # Step 4: Compile the results into a dictionary
        results_dict = {
            'jet_lat_coefficients': lat_results.params.to_dict(),
            'jet_lat_pvalues': lat_results.pvalues.to_dict(),
            'jet_lat_summary': lat_results.summary().as_text(),
            'jet_strength_coefficients': strength_results.params.to_dict(),
            'jet_strength_pvalues': strength_results.pvalues.to_dict(),
            'jet_strength_summary': strength_results.summary().as_text()
        }

        return results_dict


def detrend_timeseries(da):
    """
    Remove the linear trend from an xarray DataArray.

    Parameters:
    - da: xarray.DataArray
        The input time series to detrend.

    Returns:
    - detrended_da: xarray.DataArray
        The detrended time series.
    """
    # Check if the input is a DataArray
    if not isinstance(da, xr.DataArray):
        raise TypeError("Input must be an xarray DataArray")

    # Ensure that there is a 'time' coordinate
    if 'time' not in da.coords:
        raise ValueError("DataArray must have a 'time' coordinate")

    # Detrend the data along the time axis
    detrended_data = detrend(da, axis=0)

    # Return as a new DataArray, preserving the original coordinates
    detrended_da = xr.DataArray(detrended_data, dims=da.dims, coords=da.coords)
    
    return detrended_da


def multiple_linear_regression(target, predictors_dict):
    """
    Perform a multiple linear regression on a target time series using a dictionary of predictor time series.

    Parameters:
    - target: xarray.DataArray
        The target time series to predict.
    - predictors_dict: dict
        A dictionary where keys are predictor names and values are xarray.DataArray objects representing predictor time series.

    Returns:
    - results: statsmodels.regression.linear_model.RegressionResultsWrapper
        The results of the regression, including coefficients, p-values, etc.
    """
    # Check if input is a DataArray
    if not isinstance(target, xr.DataArray):
        raise TypeError("Target must be an xarray DataArray")
    
    # Ensure that there is a 'time' coordinate
    if 'time' not in target.coords:
        raise ValueError("Target DataArray must have a 'time' coordinate")

    # Convert the target and predictors to a pandas DataFrame
    df = pd.DataFrame({name: da.to_series() for name, da in predictors_dict.items()})
    
    # Ensure the target time series is aligned with predictors
    df['target'] = target.to_series()
    
    # Drop any rows with NaN values
    df.dropna(inplace=True)

    # Separate the predictors and the target
    X = df.drop(columns='target')
    y = df['target']

    # Add a constant (intercept) to the predictors
    X = sm.add_constant(X)

    # Perform the OLS regression
    model = sm.OLS(y, X)
    results = model.fit()

    return results


def standardize_data(da):
    """
    Standardize an xarray DataArray by subtracting the mean and dividing by the standard deviation.
    
    Parameters:
    da (xarray.DataArray or xarray.Dataset): Input data to standardize.
    
    Returns:
    xarray.DataArray or xarray.Dataset: Standardized data.
    """
    mean = da.mean(dim='time')
    std_dev = da.std(dim='time')
    
    standardized_da = (da - mean) / std_dev
    return standardized_da


def stand_detr(dato):
    anom = (dato - np.mean(dato))/np.std(dato)
    return signal.detrend(anom)

def filtro(dato):
    """Apply a rolling mean of 5 years and remov the NaNs resulting bigining and end"""
    signal = dato - dato.rolling(time=10, center=True).mean()
    signal_out = signal.dropna('time', how='all')
    return signal_out
                          
def stand(dato):
    anom = (dato - np.mean(dato))/np.std(dato)
    return anom

def replace_nans_with_zero(x):
    return np.where(np.isnan(x), random.random(), x)

def figure(target,predictors):
    fig = plt.figure()
    y = predictors.apply(stand_detr,axis=0).values
    for i in range(len(predictors.keys())):
        plt.plot(y[:,i])
    plt.plot(stand_detr(target))
    return fig

def jet_lat_strength(jet_data,lon1=140,lon2=295):
    jet_30_70 = jet_data.sel(lat=slice(-30,-70)).sel(lon=slice(lon1,lon2)).mean(dim='lon')
    lat = jet_30_70.lat
    jet_lat = (jet_30_70*lat).sum(dim='lat')/(jet_30_70).sum(dim='lat')
    strength = []
    for t,max_lat in zip(jet_data.time,jet_lat):
        strength.append(jet_data.sel(time=t).sel(lat=max_lat,method='nearest').sel(lon=slice(lon1,lon2)).mean(dim='lon'))
    jet_strength = np.array(strength)
    return np.array(jet_lat.values),jet_strength

def jet_lat_strength_model(jet_data,lon1=140,lon2=295):
    jet_30_70 = jet_data.sel(lat=slice(-70,-30)).sel(lon=slice(lon1,lon2)).mean(dim='lon')
    lat = jet_30_70.lat
    jet_lat = (jet_30_70*lat).sum(dim='lat')/(jet_30_70).sum(dim='lat')
    strength = []
    for t,max_lat in zip(jet_data.time,jet_lat):
        strength.append(jet_data.sel(time=t).sel(lat=max_lat,method='nearest').sel(lon=slice(lon1,lon2)).mean(dim='lon'))
    jet_strength = np.array(strength)
    return np.array(jet_lat.values),jet_strength

def seasonal_data(data,season='DJF'):
    # select DJF
    DA_DJF = data.sel(time = data.time.dt.season==season)

    # calculate mean per year
    DA_DJF = DA_DJF.groupby(DA_DJF.time.dt.year).mean("time")
    DA_DJF = DA_DJF.rename({'year':'time'})
    return DA_DJF

def seasonal_data_months(data, months):
    """
    Selects specified months from an xarray object and averages the data for those months within each year.
    
    Parameters:
    - data: xarray.DataArray or xarray.Dataset
        The input data to process. It should have a 'time' coordinate.
    - months: list of int
        The months to select for averaging (1 = January, 2 = February, ..., 12 = December).
    
    Returns:
    - xarray.DataArray or xarray.Dataset
        The averaged data for the selected months within each year, accounting for months that span across years.
    """
    # Ensure 'time' coordinate is in a format that supports .dt accessor
    if np.issubdtype(data['time'].dtype, np.datetime64):
        time_coord = data['time']
    else:
        time_coord = xr.cftime_range(start=data['time'][0].values, periods=data['time'].size, freq='M')
        data = data.assign_coords(time=time_coord)

    # Select the relevant months and keep track of the original years
    selected_months_data = data.sel(time=data['time'].dt.month.isin(months))

    # Create a new time coordinate for grouping
    new_years = selected_months_data['time'].dt.year.values.copy()

    # Shift the year for December, if necessary
    if 12 in months:
        dec_mask = selected_months_data['time'].dt.month == 12
        new_years[dec_mask] += 1  # Increment year for December

    # Assign the new year as a coordinate to the selected data
    selected_months_data = selected_months_data.assign_coords(new_year=("time", new_years))

    # Now group by the new year and calculate the mean
    averaged_data = selected_months_data.groupby("new_year").mean(dim="time")

    # Rename the new year dimension to 'time' for consistency
    averaged_data = averaged_data.rename({"new_year": "time"})

    return averaged_data


#Across models regression class
class spatial_MLR(object):
    def __init__(self):
        self.what_is_this = 'This performs a regression across models and plots everything'
    
    def regression_data(self,variable,regressors,regressor_names,dataset):
        """Define the regression target variable 
        this is here to be edited if some opperation is needed on the DataArray
        
        :param variable: DataArray
        :return: target variable for the regression  
        """
        self.dataset = dataset
        self.target = variable
        regressor_indices = regressors
        self.regression_y = sm.add_constant(regressors.values)
        self.regressors = regressors.values
        self.rd_num = len(regressor_names) 
        self.regressor_names = regressor_names

    #Regresion lineal
    def linear_regression(self,x):
        y = self.regression_y
        res = sm.OLS(x,y).fit()
        returns = [res.params[i] for i in range(self.rd_num)]
        return tuple(returns)

    def linear_regression_pvalues(self,x):
        y = self.regression_y
        res = sm.OLS(x,y).fit()
        returns = [res.pvalues[i] for i in range(self.rd_num)]
        return tuple(returns)
    
    def linear_regression_R2(self,x):
        y = self.regression_y
        res = sm.OLS(x,y).fit()
        return res.rsquared
    

    def perform_regression(self,path,var): 
        """ Performs regression over all gridpoints in a map and returns and saves DataFrames
        
        :param path: saving path
        :return: none
        """
        
        target_var = xr.apply_ufunc(replace_nans_with_zero, self.target)
        results = xr.apply_ufunc(self.linear_regression,target_var,input_core_dims=[["time"]],
                                 output_core_dims=[[] for i in range(self.rd_num)],
                                 vectorize=True,
                                 dask="parallelized")
        results_pvalues = xr.apply_ufunc(self.linear_regression_pvalues,target_var,input_core_dims=[["time"]],
                                 output_core_dims=[[] for i in range(self.rd_num)],
                                 vectorize=True,
                                 dask="parallelized")
        results_R2 = xr.apply_ufunc(self.linear_regression_R2,target_var,input_core_dims=[["time"]],
                                 output_core_dims=[[]],
                                 vectorize=True,
                                 dask="parallelized")
        
      
        for i in range(self.rd_num):
            if i == 0:
                regression_coefs = results[0].to_dataset(name='const')
            else:
                regression_coefs[self.regressor_names[i]] = results[i]
                
        print('This is regressor_coefs:',regression_coefs)
        if var == 'ua':
            regression_coefs = regression_coefs.rename({'ua':self.regressor_names[0]})
        elif var == 'sst':
            regression_coefs = regression_coefs.rename({'tos':self.regressor_names[0]})
        elif var == 'tas':
            regression_coefs = regression_coefs.rename({'tas':self.regressor_names[0]})
        elif var == 'pr':
            regression_coefs = regression_coefs.rename({'pr':self.regressor_names[0]})
        else:
            'done'
            #regression_coefs = regression_coefs.rename({var:self.regressor_names[0]})
        regression_coefs.to_netcdf(path+'/'+var+'/regression_coefficients_'+self.dataset+'.nc')
        
        for i in range(self.rd_num):
            if i == 0:
                regression_coefs_pvalues = results_pvalues[0].to_dataset(name='const')
            else:
                regression_coefs_pvalues[self.regressor_names[i]] = results_pvalues[i]        
        if var == 'ua':
            regression_coefs_pvalues = regression_coefs_pvalues.rename({'ua':self.regressor_names[0]})
        elif var == 'sst':
            regression_coefs_pvalues = regression_coefs_pvalues.rename({'tos':self.regressor_names[0]})
        elif var == 'tas':
            regression_coefs_pvalues = regression_coefs_pvalues.rename({'tas':self.regressor_names[0]})
        elif var == 'pr':
            regression_coefs_pvalues = regression_coefs_pvalues.rename({'pr':self.regressor_names[0]})
        else:
            'done'
            #regression_coefs_pvalues = regression_coefs_pvalues.rename({var:self.regressor_names[0]})
        regression_coefs_pvalues.to_netcdf(path+'/'+var+'/regression_coefficients_pvalues_'+self.dataset+'.nc')
        

        results_R2.to_netcdf(path+'/'+var+'/R2_'+self.dataset+'.nc')
                     
        
    def create_x(self,i,j,dato):
        """ For each gridpoint creates an array and standardizes it 
        :param regressor_names: list with strings naming the independent variables
        :param path: saving path
        :return: none
        """    
        x = np.array([])
        for y in range(len(dato.time)):
            aux = dato.isel(time=y)
            x = np.append(x,aux[i-1,j-1].values)
        return stand(x)
     
    
    def open_regression_coef(self,path,var,dataset):
        """ Open regression coefficients and pvalues to plot
        :param path: saving path
        :return maps: list of list of coefficient maps
        :return maps_pval:  list of coefficient pvalues maps
        :return R2: map of fraction of variance
        """ 
        maps = []; maps_pval = []
        coef_maps = xr.open_dataset(path+'/'+var+'/regression_coefficients_'+dataset+'.nc')
        coef_pvalues = xr.open_dataset(path+'/'+var+'/regression_coefficients_pvalues_'+dataset+'.nc')
        maps = [coef_maps[variable] for variable in self.regressor_names]
        maps_pval = [coef_pvalues[variable] for variable in self.regressor_names]
        R2 = xr.open_dataset(path+'/'+var+'/R2_'+dataset+'.nc')
        return maps, maps_pval, R2    

    def open_lmg_coef(self,path,var):
        """ Open regression coefficients and pvalues to plot
        :param path: saving path
        :return maps: list of list of coefficient maps
        :return maps_pval:  list of coefficient pvalues maps
        :return R2: map of fraction of variance
        """ 
        maps = []; maps_pval = []
        coef_maps = xr.open_dataset(path+'/'+var+'/regression_coefficients_relative_importance.nc')
        coef_pvalues = xr.open_dataset(path+'/'+var+'/regression_coefficients_pvalues.nc')
        maps = [coef_maps[variable] for variable in self.regressor_names[1:]]
        maps_pval = [coef_pvalues[variable] for variable in self.regressor_names]
        R2 = xr.open_dataset(path+'/'+var+'/R2.nc')
        return maps, maps_pval, R2    
    
    def plot_regression_lmg_map(self,path,var,output_path):
        """ Plots figure with all of 
        :param regressor_names: list with strings naming the independent variables
        :param path: saving path
        :return: none
        """
        maps, maps_pval, R2 = self.open_lmg_coef(path,var)
        cmapU850 = mpl.colors.ListedColormap(['darkblue','navy','steelblue','lightblue',
                                            'lightsteelblue','white','white','mistyrose',
                                            'lightcoral','indianred','brown','firebrick'])
        cmapU850.set_over('maroon')
        cmapU850.set_under('midnightblue')
        path_era = '/datos/ERA5/mon'
        u_ERA = xr.open_dataset(path_era+'/era5.mon.mean.nc')
        u_ERA = u_ERA.u.sel(lev=850).sel(time=slice('1979','2018'))
        u_ERA = u_ERA.groupby('time.season').mean(dim='time').sel(season='DJF')

        fig_coef = plt.figure(figsize=(20, 16),dpi=100,constrained_layout=True)
        projection_stereo = ccrs.SouthPolarStereo(central_longitude=300)
        projection_plate = ccrs.PlateCarree(180)
        data_crs = ccrs.PlateCarree()
        for k in range(self.rd_num-1):
            lat = maps[k].lat
            lon = np.linspace(0,360,len(maps[k].lon))
            var_c, lon_c = add_cyclic_point(maps[k].values,lon)
            #SoutherHemisphere Stereographic
            if var == 'ua':
                ax = plt.subplot(3,3,k+1,projection=projection_stereo)
                ax.set_extent([0,359.9, -90, 0], crs=data_crs)
                theta = np.linspace(0, 2*np.pi, 100)
                center, radius = [0.5, 0.5], 0.5
                verts = np.vstack([np.sin(theta), np.cos(theta)]).T
                circle = mpath.Path(verts * radius + center)
                ax.set_boundary(circle, transform=ax.transAxes)
            elif var == 'sst':
                ax = plt.subplot(3,3,k+1,projection=projection_plate)
            else: 
                ax = plt.subplot(3,3,k+1,projection=projection_stereo)
            clevels = np.arange(0,40,2)
            im=ax.contourf(lon_c, lat, var_c*100,clevels,transform=data_crs,cmap='OrRd',extend='both')
            cnt=ax.contour(u_ERA.lon,u_ERA.lat, u_ERA.values,levels=[8],transform=data_crs,linewidths=1.2, colors='black', linestyles='-')
            plt.clabel(cnt,inline=True,fmt='%1.0f',fontsize=8)
            if maps_pval[k+1].min() < 0.05: 
                levels = [maps_pval[k+1].min(),0.05,maps_pval[k+1].max()]
                ax.contourf(maps_pval[k+1].lon, lat, maps_pval[k+1].values,levels, transform=data_crs,levels=levels, hatches=["...", " "], alpha=0)
            elif maps_pval[k+1].min() < 0.10:
                levels = [maps_pval[k+1].min(),0.10,maps_pval[k+1].max()]
                ax.contourf(maps_pval[k+1].lon, lat, maps_pval[k+1].values,levels, transform=data_crs,levels=levels, hatches=["...", " "], alpha=0)
            else:
                print('No significant values for ',self.regressor_names[k+1]) 
            plt.title(self.regressor_names[k+1],fontsize=18)
            ax.add_feature(cartopy.feature.COASTLINE,alpha=.5)
            ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=.5)
            ax.gridlines(crs=data_crs, linewidth=0.3, linestyle='-')
            ax.set_extent([-180, 180, -90, -25], ccrs.PlateCarree())
        plt1_ax = plt.gca()
        left, bottom, width, height = plt1_ax.get_position().bounds
        if var == 'ua':
            colorbar_axes1 = fig_coef.add_axes([left+0.5, bottom, 0.01, height*2])
        elif var == 'sst':
            colorbar_axes1 = fig_coef.add_axes([left+0.3, bottom, 0.01, height*2])    
        else:
            colorbar_axes1 = fig_coef.add_axes([left+0.5, bottom, 0.01, height*2])
        cbar = fig_coef.colorbar(im, colorbar_axes1, orientation='vertical')
        cbar.set_label('relative importance',fontsize=14) #rotation = radianes
        cbar.ax.tick_params(axis='both',labelsize=14)
            
        plt.subplots_adjust(bottom=0.2, right=.95, top=0.8)
        if var == 'ua':
            plt.savefig(output_path+'/regression_coefficients_relative_importance_u850',bbox_inches='tight')
        elif var == 'sst':
            plt.savefig(output_path+'/regression_coefficients_relative_importance_sst',bbox_inches='tight')
        else:
            plt.savefig(output_path+'/regression_coefficients_relative_importance_XXX',bbox_inches='tight')   
        plt.clf

        return fig_coef


    def plot_regression_coef_map(self,path,var,output_path):
        """ Plots figure with all of 
        :param regressor_names: list with strings naming the independent variables
        :param path: saving path
        :return: none
        """
        maps, maps_pval, R2 = self.open_regression_coef(path,var,self.dataset)
        cmapU850 = mpl.colors.ListedColormap(['darkblue','navy','steelblue','lightblue',
                                            'lightsteelblue','white','white','mistyrose',
                                            'lightcoral','indianred','brown','firebrick'])
        cmapU850.set_over('maroon')
        cmapU850.set_under('midnightblue')
        u_ERA = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5/ua_ERA5.nc')
        u_ERA = u_ERA.u.sel(level=850).sel(time=slice('1979','2018'))
        u_ERA = u_ERA.rename({'longitude':'lon','latitude':'lat'})
        u_ERA = u_ERA.groupby('time.season').mean(dim='time').sel(season='DJF')

        fig_coef = plt.figure(figsize=(20, 16),dpi=100,constrained_layout=True)
        projection_stereo = ccrs.SouthPolarStereo(central_longitude=300)
        projection_plate = ccrs.PlateCarree(180)
        data_crs = ccrs.PlateCarree()
        for k in range(self.rd_num):
            lat = maps[k].lat
            lon = np.linspace(0,360,len(maps[k].lon))
            var_c, lon_c = add_cyclic_point(maps[k].values,lon)
            #SoutherHemisphere Stereographic for winds
            if var == 'u':
                ax = plt.subplot(3,2,k+1,projection=projection_stereo)
                ax.set_extent([0,359.9, -90, 0], crs=data_crs)
                theta = np.linspace(0, 2*np.pi, 100)
                center, radius = [0.5, 0.5], 0.5
                verts = np.vstack([np.sin(theta), np.cos(theta)]).T
                circle = mpath.Path(verts * radius + center)
                ax.set_boundary(circle, transform=ax.transAxes)
            #Plate Carree map for SST
            elif var == 'sst':
                ax = plt.subplot(3,2,k+1,projection=projection_plate)
            else: 
                ax = plt.subplot(3,2,k+1,projection=projection_stereo)
            if k == 6:
                im0=ax.contourf(lon_c, lat, var_c,transform=data_crs,cmap='OrRd',extend='both')
            else:
                clevels = np.arange(-.6,.7,0.1)
                im=ax.contourf(lon_c, lat, var_c,clevels,transform=data_crs,cmap='RdBu_r',extend='both')
            cnt=ax.contour(u_ERA.lon,u_ERA.lat, u_ERA.values,levels=[8],transform=data_crs,linewidths=1.2, colors='black', linestyles='-')
            plt.clabel(cnt,inline=True,fmt='%1.0f',fontsize=8)
            if maps_pval[k].min() < 0.05: 
                try:
                    levels = [maps_pval[k].min(),0.05,maps_pval[k].max()]
                    ax.contourf(maps_pval[k].lon, lat, maps_pval[k].values,levels, transform=data_crs,levels=levels, hatches=["...", " "], alpha=0)
                except:
                    levels = [maps_pval[k].min(),maps_pval[k].min()+0.01*maps_pval[k].min(),maps_pval[k].max()]
                    ax.contourf(maps_pval[k].lon, lat, maps_pval[k].values,levels, transform=data_crs,levels=levels, hatches=["...", " "], alpha=0)
            elif maps_pval[k].min() < 0.50:
                levels = [maps_pval[k].min(),0.50,maps_pval[k].max()]
                ax.contourf(maps_pval[k].lon, lat, maps_pval[k].values,levels, transform=data_crs,levels=levels, hatches=["...", " "], alpha=0)
            else:
                print('No significant values for ',self.regressor_names[k]) 
            plt.title(self.regressor_names[k],fontsize=18)
            ax.add_feature(cartopy.feature.COASTLINE,alpha=.5)
            ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=.5)
            ax.gridlines(crs=data_crs, linewidth=0.3, linestyle='-')
            if var == 'ua':
                ax.set_extent([-180, 180, -90, -25], ccrs.PlateCarree())
            elif var == 'sst':
                ax.set_extent([-60, 220, -80, 40], ccrs.PlateCarree(central_longitude=180))
            else: 
                ax.set_extent([-60, 220, -80, -25], ccrs.PlateCarree(central_longitude=180))
            
        plt1_ax = plt.gca()
        left, bottom, width, height = plt1_ax.get_position().bounds
        if var == 'ua':
            colorbar_axes1 = fig_coef.add_axes([left+0.28, bottom, 0.01, height*2])
            colorbar_axes2 = fig_coef.add_axes([left+0.36, bottom, 0.01, height*2])
        elif var == 'sst':
            colorbar_axes1 = fig_coef.add_axes([left+0.3, bottom, 0.01, height*3])
            colorbar_axes2 = fig_coef.add_axes([left+0.38, bottom, 0.01, height*3])
        else:
            colorbar_axes1 = fig_coef.add_axes([left+0.28, bottom, 0.01, height*2])
            colorbar_axes2 = fig_coef.add_axes([left+0.36, bottom, 0.01, height*2])
        cbar = fig_coef.colorbar(im, colorbar_axes1, orientation='vertical')
        cbar2 = fig_coef.colorbar(im, colorbar_axes2, orientation='vertical')
        if var == 'ua':
            cbar.set_label('m/s/std(rd)',fontsize=14) #rotation = radianes
            cbar2.set_label('m/s/std(rd)',fontsize=14) #rotation = radianes
        elif var == 'sst':
            cbar.set_label('K/std(rd)',fontsize=14) #rotation = radianes
            cbar2.set_label('K/std(rd)',fontsize=14) #rotation = radianes
        else:
            cbar.set_label('X/std(rd)',fontsize=14) #rotation = radianes
            cbar2.set_label('X/std(rd)',fontsize=14) #rotation = radianes
        cbar.ax.tick_params(axis='both',labelsize=14)
        cbar2.ax.tick_params(axis='both',labelsize=14)
            
        plt.subplots_adjust(bottom=0.2, right=.95, top=0.8)
        if var == 'ua':
            plt.savefig(output_path+'/regression_coefficients_u850',bbox_inches='tight')
        elif  var == 'sst':
            plt.savefig(output_path+'/regression_coefficients_sst',bbox_inches='tight')
        else:
            plt.savefig(output_path+'/regression_coefficients_unknown_var',bbox_inches='tight')
        
        plt.clf

        return fig_coef


def plot_regression_coef_map_MEM(maps, maps_pval, regressor_names, output_path):
    """Plots figure with regression coefficient maps with two distinct colorbars.
    :param regressor_names: list with strings naming the independent variables
    :param path: saving path
    :return: figure
    """

    # Custom colormap
    cmapU850 = mpl.colors.ListedColormap(['darkblue', 'navy', 'steelblue', 'lightblue',
                                'lightsteelblue', 'white', 'white', 'mistyrose',
                                'lightcoral', 'indianred', 'brown', 'firebrick'])
    cmapU850.set_over('maroon')
    cmapU850.set_under('midnightblue')

    # Load data for contours
    ua_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5/ua_ERA5.nc')
    ua_era5 = ua_era5.rename({'latitude': 'lat', 'longitude': 'lon'})
    ua_era5_850 = ua_era5.sel(level=850)
    u_ERA = ua_era5_850.u.sel(time=slice('1979', '2018'))
    u_ERA = u_ERA.groupby('time.season').mean(dim='time').sel(season='DJF')

    # Create figure and subplots with adjusted size
    fig_coef, axs = plt.subplots(2, 3, figsize=(24, 15), dpi=100,
                                subplot_kw={'projection': ccrs.SouthPolarStereo(central_longitude=300)})
    plt.subplots_adjust(bottom=0.1, right=0.85, top=0.85, hspace=0.1, wspace=0.25)

    data_crs = ccrs.PlateCarree()

    # Loop over the subplots
    for k, ax in enumerate(axs.flat):
        if k >= len(maps):  # Stop if we have more subplots than data
            break
        
        lat = maps[k].lat
        lon = np.linspace(0, 360, len(maps[k].lon))
        var_c, lon_c = add_cyclic_point(maps[k].values, lon)

        ax.set_extent([0, 359.9, -90, 0], crs=data_crs)
        theta = np.linspace(0, 2 * np.pi, 100)
        center, radius = [0.5, 0.5], 0.5
        verts = np.vstack([np.sin(theta), np.cos(theta)]).T
        circle = mpath.Path(verts * radius + center)
        ax.set_boundary(circle, transform=ax.transAxes)

        # Use different color scales for the first and other subplots
        if k == 0:
            clevels = np.arange(0, 40, 5)
            # Contour plot
            im0 = ax.contourf(lon_c, maps[k].lat, var_c, clevels, transform=data_crs, cmap='OrRd', extend='both') #maps[k].values
        else:
            clevels = np.arange(-1, 1.1, 0.1)
            # Contour plot
            try:
                im = ax.contourf(lon_c, maps[k].lat, var_c, clevels, transform=data_crs, cmap=cmapU850, extend='both')
            except TypeError:
                print(maps[k])

        # Overlay contour lines for u_ERA
        cnt = ax.contour(u_ERA.lon, u_ERA.lat, u_ERA.values, levels=[8], transform=data_crs,
                        linewidths=1.2, colors='black', linestyles='-')
        ax.clabel(cnt, inline=True, fmt='%1.0f', fontsize=8)

        # Check for significant p-values and hatch regions
        if (maps_pval[k].min() < 0.05) & (k > 0):
            try:
                levels = [maps_pval[k].min(), 0.05, maps_pval[k].max()]
                ax.contourf(maps_pval[k].lon, lat, maps_pval[k].values, levels=levels,
                            transform=data_crs, hatches=["...", " "], alpha=0)
            except:
                levels = [maps_pval[k].min(), maps_pval[k].min()+maps_pval[k].min()*0.01, maps_pval[k].max()]
                ax.contourf(maps_pval[k].lon, lat, maps_pval[k].values, levels=levels,
                            transform=data_crs, hatches=["...", " "], alpha=0)
        

        # Define box coordinates (example: covers part of North America)
        lon_min, lon_max = 0, 360
        lat_min, lat_max = -70, -30
        width = lon_max - lon_min
        height = lat_max - lat_min

        # Create a rectangle patch
        rect = patches.Rectangle((lon_min, lat_min), width, height,
                                linewidth=1, edgecolor='black', facecolor='none')

        # Add rectangle to the map
        ax.add_patch(rect)

        # Define box coordinates (example: covers part of North America)
        lon_min, lon_max = 120, 180
        lat_min, lat_max = -70, -30
        width = lon_max - lon_min
        height = lat_max - lat_min

        # Create a rectangle patch
        rect = patches.Rectangle((lon_min, lat_min), width, height,
                                linewidth=1, edgecolor='red', facecolor='none')

        # Add rectangle to the map
        ax.add_patch(rect)

        # Plot title
        ax.set_title(regressor_names[k], fontsize=20)
        
        # Add coastlines and borders
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
        ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=.5)
        ax.gridlines(crs=data_crs, linewidth=0.3, linestyle='-')
        ax.set_extent([-180, 180, -90, -25], ccrs.PlateCarree())


    # Create two colorbars outside the grid of subplots

    # Colorbar for the first subplot
    cbar_ax_1 = fig_coef.add_axes([0.87, 0.55, 0.02, 0.25])  # Manually specify position
    cbar_1 = fig_coef.colorbar(im0, cax=cbar_ax_1, orientation='vertical', ticks=np.arange(0, 40, 5))
    cbar_1.set_label(r'${R}^{2}$', fontsize=16)
    cbar_1.ax.tick_params(axis='both', labelsize=15)

    # Add "panel a" text above the first colorbar
    plt.text(0.87, 0.82, 'panel a', fontsize=16, transform=fig_coef.transFigure, ha='center')

    # Colorbar for the remaining subplots
    cbar_ax_2 = fig_coef.add_axes([0.87, 0.18, 0.02, 0.25])  # Manually specify position
    cbar_2 = fig_coef.colorbar(im, cax=cbar_ax_2, orientation='vertical', ticks=np.arange(-1, 1.2, 0.2))
    cbar_2.set_label(r'$\sigma$ $\sigma_{RD}^{-1}$', fontsize=16)
    cbar_2.ax.tick_params(axis='both', labelsize=18)

    # Add "panels b-f" text above the second colorbar
    plt.text(0.87, 0.45, 'panels b-f', fontsize=15, transform=fig_coef.transFigure, ha='center')

    plt.savefig(output_path, bbox_inches='tight')
    plt.clf()

    return fig_coef


def plot_regression_coef_map_row(maps, maps_pval, regressor_names, output_path):
    """Plots figure with regression coefficient maps with two distinct colorbars.
    :param regressor_names: list with strings naming the independent variables
    :param path: saving path
    :return: figure
    """

    # Custom colormap
    cmapU850 = mpl.colors.ListedColormap(['darkblue', 'navy', 'steelblue', 'lightblue',
                                'lightsteelblue', 'white', 'white', 'mistyrose',
                                'lightcoral', 'indianred', 'brown', 'firebrick'])
    cmapU850.set_over('maroon')
    cmapU850.set_under('midnightblue')

    # Load data for contours
    ua_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5/ua_ERA5.nc')
    ua_era5 = ua_era5.rename({'latitude': 'lat', 'longitude': 'lon'})
    ua_era5_850 = ua_era5.sel(level=850)
    u_ERA = ua_era5_850.u.sel(time=slice('1979', '2018'))
    u_ERA = u_ERA.groupby('time.season').mean(dim='time').sel(season='DJF')

    # Create figure and subplots with adjusted size
    fig_coef, axs = plt.subplots(1, 6, figsize=(30, 10), dpi=100,
                                subplot_kw={'projection': ccrs.SouthPolarStereo(central_longitude=300)})
    plt.subplots_adjust(bottom=0.1, right=0.85, top=0.85, hspace=0.1, wspace=0.25)

    data_crs = ccrs.PlateCarree()

    # Loop over the subplots
    for k, ax in enumerate(axs.flat):
        if k >= len(maps):  # Stop if we have more subplots than data
            break
        
        lat = maps[k].lat
        lon = np.linspace(0, 360, len(maps[k].lon))
        var_c, lon_c = add_cyclic_point(maps[k].values, lon)

        ax.set_extent([0, 359.9, -90, 0], crs=data_crs)
        theta = np.linspace(0, 2 * np.pi, 100)
        center, radius = [0.5, 0.5], 0.5
        verts = np.vstack([np.sin(theta), np.cos(theta)]).T
        circle = mpath.Path(verts * radius + center)
        ax.set_boundary(circle, transform=ax.transAxes)

        # Use different color scales for the first and other subplots
        if k == 0:
            clevels = np.arange(0, 35, 5)
            # Contour plot
            im0 = ax.contourf(lon_c, maps[k].lat, var_c, clevels, transform=data_crs, cmap='OrRd', extend='both') #maps[k].values
        else:
            clevels = np.arange(-1, 1.1, 0.1)
            # Contour plot
            try:
                im = ax.contourf(lon_c, maps[k].lat, var_c, clevels, transform=data_crs, cmap=cmapU850, extend='both')
            except TypeError:
                print(maps[k])

        # Overlay contour lines for u_ERA
        cnt = ax.contour(u_ERA.lon, u_ERA.lat, u_ERA.values, levels=[8], transform=data_crs,
                        linewidths=1.2, colors='black', linestyles='-')
        ax.clabel(cnt, inline=True, fmt='%1.0f', fontsize=8)

        # Check for significant p-values and hatch regions
        if (maps_pval[k].min() < 0.05) & (k > 0):
            try:
                levels = [maps_pval[k].min(), 0.05, maps_pval[k].max()]
                ax.contourf(maps_pval[k].lon, lat, maps_pval[k].values, levels=levels,
                            transform=data_crs, hatches=["...", " "], alpha=0)
            except:
                levels = [maps_pval[k].min(), maps_pval[k].min()+maps_pval[k].min()*0.01, maps_pval[k].max()]
                ax.contourf(maps_pval[k].lon, lat, maps_pval[k].values, levels=levels,
                            transform=data_crs, hatches=["...", " "], alpha=0)
        

        # Define box coordinates (example: covers part of North America)
        lon_min, lon_max = 0, 360
        lat_min, lat_max = -70, -30
        width = lon_max - lon_min
        height = lat_max - lat_min

        # Create a rectangle patch
        rect = patches.Rectangle((lon_min, lat_min), width, height,
                                linewidth=1, edgecolor='black', facecolor='none')

        # Add rectangle to the map
        ax.add_patch(rect)

        # Define box coordinates (example: covers part of North America)
        lon_min, lon_max = 120, 180
        lat_min, lat_max = -70, -30
        width = lon_max - lon_min
        height = lat_max - lat_min

        # Create a rectangle patch
        rect = patches.Rectangle((lon_min, lat_min), width, height,
                                linewidth=1, edgecolor='red', facecolor='none')

        # Add rectangle to the map
        ax.add_patch(rect)

        # Plot title
        ax.set_title(regressor_names[k], fontsize=20)
        
        # Add coastlines and borders
        ax.add_feature(cartopy.feature.COASTLINE, alpha=.5)
        ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=.5)
        ax.gridlines(crs=data_crs, linewidth=0.3, linestyle='-')
        ax.set_extent([-180, 180, -90, -25], ccrs.PlateCarree())


    # Create two colorbars outside the grid of subplots

    # Colorbar for the first subplot
    #cbar_ax_1 = fig_coef.add_axes([0.8, 0.55, 0.02, 0.25])  # Manually specify position
    cbar_1 = fig_coef.colorbar(im0, orientation='horizontal', ticks=np.arange(0, 40, 5))
    cbar_1.set_label(r'${R}^{2}$', fontsize=16)
    cbar_1.ax.tick_params(axis='both', labelsize=15)

    # Colorbar for the remaining subplots
    #cbar_ax_2 = fig_coef.add_axes([0.88, 0.55, 0.02, 0.25])  # Manually specify position
    cbar_2 = fig_coef.colorbar(im, orientation='horizontal', ticks=np.arange(-1, 1.2, 0.2))
    cbar_2.set_label(r'$\sigma$ $\sigma_{RD}^{-1}$', fontsize=16)
    cbar_2.ax.tick_params(axis='both', labelsize=18)

    plt.savefig(output_path, bbox_inches='tight')
    plt.clf()

    return fig_coef



#### Load data

In [7]:
### Import ERA5 data

ua_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5/ua_ERA5.nc')
ua_era5 = ua_era5.rename({'latitude':'lat','longitude':'lon'})
ua_era5_50 = ua_era5.sel(level=50)
ua_era5_850 = ua_era5.sel(level=850)
ta_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5/ta_ERA5.nc')
ta_era5 = ta_era5.rename({'latitude':'lat','longitude':'lon'})

### Import NCEP data

ua_ncep = xr.open_dataset('/home/jmindlin/causal_EDJ/NCEP/uwnd.mon.mean.nc')
ua_ncep_50 = ua_ncep.sel(level=50)
ua_ncep_850 = ua_ncep.sel(level=850)
ta_ncep = xr.open_dataset('/home/jmindlin/causal_EDJ/NCEP/air.mon.mean.nc')

del ua_era5, ua_ncep

### Import JRA55 data

ua_jra55_50 = [xr.open_dataset('/home/jmindlin/causal_EDJ/JRA55/ua/anl_mdl.033_ugrd.reg_tl319.'+str(year)+'01_'+str(year)+'12.mindlin756630_50hPa.nc') for year in np.arange(1958,2024,1)]
ua_jra55_50_concat = xr.concat(ua_jra55_50,dim='initial_time0_hours')
ua_jra55_50_concat = ua_jra55_50_concat.rename({'initial_time0_hours':'time','g4_lat_2':'lat','g4_lon_3':'lon'})

ua_jra55_850 = [xr.open_dataset('/home/jmindlin/causal_EDJ/JRA55/ua/anl_mdl.033_ugrd.reg_tl319.'+str(year)+'01_'+str(year)+'12.mindlin756630_847hPa.nc') for year in np.arange(1958,2024,1)]
ua_jra55_850_concat = xr.concat(ua_jra55_850,dim='initial_time0_hours')
ua_jra55_850_concat = ua_jra55_850_concat.rename({'initial_time0_hours':'time','g4_lat_2':'lat','g4_lon_3':'lon'})

ta_jra55 = [xr.open_dataset('/home/jmindlin/causal_EDJ/JRA55/ta/anl_mdl.011_tmp.reg_tl319.'+str(year)+'01_'+str(year)+'12.mindlin754486.nc') for year in np.arange(1958,2024,1)]
ta_jra55_concat = xr.concat(ta_jra55,dim='initial_time0_hours')
ta_jra55_concat = ta_jra55_concat.rename({'initial_time0_hours':'time','g4_lat_2':'lat','g4_lon_3':'lon','lv_HYBL1':'lev'})

In [4]:
def regressor_EESC_GW(gw_ts):
    eesc_ts = pd.read_csv('/home/jmindlin/causal_EDJ/send_to_LIM/GW_EESC_polar_ozoneloss.csv')
    for i in range(len(eesc_ts[:8])):
        eesc_ts['EESC_polar'][i] = eesc_ts['EESC_polar'][8]
    df = pd.DataFrame({'EESC':eesc_ts['EESC_polar'][10:79] - eesc_ts['EESC_polar'][8],'GW':gw_ts})
    regressors_out = sm.add_constant(df.values)
    return regressors_out, df

import urllib.request

# URL of the data file
url = "https://crudata.uea.ac.uk/cru/data/temperature/HadCRUT5.0Analysis_gl.txt"

# Fetch the data from the URL
with urllib.request.urlopen(url) as response:
    lines = response.read().decode('utf-8').splitlines()

# Parse the lines to extract the data
data = []
months = []
years = []
for line in lines[::2]:
    values = line.split(' ')[2:-1]
    years.append(line.split(' ')[1])
    for i, value in enumerate(values):
        if value != '':
            data.append(value)
            months.append(i)

# Convert the list of lists into a NumPy array
data_array = np.array(data, dtype=float)
data_array = data_array

# Print the resulting NumPy array
print(data_array)

time = pd.date_range(start='1850-01-01', end='2024-12-01', freq='MS')
temperature_data = xr.DataArray(
    data_array, 
    coords={'time': time}, 
    dims='time', 
    name='temperature - HadCRU5'
)

tas_DJF = seasonal_data_months(temperature_data.sel(time=slice('1950','2023')),[12,1,2])

[-0.675 -0.333 -0.591 ... -9.999 -9.999 -9.999]


In [1]:
u_1979_2019 = xr.open_dataset('/home/jmindlin/causal_EDJ/send_to_LIM/ERA5/era5.mon.mean_T42.nc').u.sel(lev=50).drop_vars('lev')
u_1950_1978 = xr.open_dataset('/home/jmindlin/causal_EDJ/send_to_LIM/ERA5/ERA5_monthly_u_wind_n36_rename_regrid.nc').u.sel(plev=5000).drop_vars('plev')
u_1950_2019 = xr.concat([u_1950_1978,u_1979_2019],'time')
spv_era5_OND = seasonal_data_months(u_1950_2019,[10,11,12]).sel(lat=slice(-50,-60)).mean(dim='lat').mean(dim='lon').sel(time=slice('1950','2019'))
spv_era5_1950_2019_OND = spv_era5_OND
CLIM_spv_era5 = spv_era5_1950_2019_OND.sel(time=slice('1950','2019')).mean(dim='time')

spv_ncep_OND = seasonal_data_months(ua_ncep_50,[10,11,12]).sel(lat=slice(-50,-60)).mean(dim='lat').mean(dim='lon').sel(time=slice('1950','2023'))
spv_ncep_1950_2023_OND = spv_ncep_OND
CLIM_spv_ncep = spv_ncep_1950_2023_OND.sel(time=slice('1950','2023')).mean(dim='time')

spv_jra55_OND = seasonal_data_months(ua_jra55_50_concat,[10,11,12]).sel(lat=slice(-50,-60)).mean(dim='lat').mean(dim='lon').sel(time=slice('1950','2023'))
spv_jra55_1950_2023_OND = spv_jra55_OND
CLIM_spv_jra55 = spv_jra55_1950_2023_OND.sel(time=slice('1950','2023')).mean(dim='time')

In [ ]:
### Extend ERA5 SPV 2019-2023 with NCEP data
spv_era5_1950_2023_OND = xr.concat([spv_era5_1950_2019_OND,spv_ncep_1950_2023_OND.uwnd.sel(time=slice('2020','2023'))],dim='time')

stratospheric_polar_vortex_rean = []
stratospheric_polar_vortex_rean.append(spv_era5_1950_2023_OND)
stratospheric_polar_vortex_rean.append(spv_ncep_1950_2023_OND)
stratospheric_polar_vortex_rean.append(spv_jra55_1950_2023_OND)

In [ ]:
tropical_warming = []
tw_era5_DJF = seasonal_data_months(ta_era5,[12,1,2]).sel(lat=slice(15,-15)).mean(dim='lat').mean(dim='lon').sel(time=slice('1950','2023'))
tw_era5_1950_2023_DJF = tw_era5_DJF
tropical_warming.append(tw_era5_DJF)
CLIM_tw_ncep = tw_era5_1950_2023_DJF.sel(time=slice('1950','2023')).mean(dim='time')

tw_ncep_DJF = seasonal_data_months(ta_ncep,[12,1,2]).sel(level=250).sel(lat=slice(15,-15)).mean(dim='lat').mean(dim='lon').sel(time=slice('1950','2023')) + 273.15
tw_ncep_1950_2023_DJF = tw_ncep_DJF
tropical_warming.append(tw_ncep_DJF)
CLIM_tw_ncep = tw_ncep_1950_2023_DJF.sel(time=slice('1950','2023')).mean(dim='time')

tw_jra55_DJF = seasonal_data_months(ta_jra55_concat,[12,1,2]).sel(lev=29).sel(lat=slice(15,-15)).mean(dim='lat').mean(dim='lon').sel(time=slice('1950','2023'))
tw_jra55_1950_2023_DJF = tw_jra55_DJF
tropical_warming.append(tw_jra55_DJF)
CLIM_tw_jra55 = tw_jra55_1950_2023_DJF.sel(time=slice('1950','2023')).mean(dim='time')

In [ ]:
### SST data
sst_ERSST = xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mnmean_ERSST_2022_KAPLAN_grid.nc') #- xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mnmean_ERSST_2022_KAPLAN_grid.nc').mean(dim='lon')
sst_COBE = xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mon.mean_COBE_2022_KAPLAN_grid.nc')# - xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mon.mean_COBE_2022_KAPLAN_grid.nc').mean(dim='lon')
sst_HadISST = xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/HadISST_sst_latest_KAPLAN_grid.nc') #- xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/HadISST_sst_latest_KAPLAN_grid.nc').mean(dim='lon')
sst_Kaplan = xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mean.anom_Kaplan_2022_KAPLAN_grid.nc') #- xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mean.anom_Kaplan_2022_KAPLAN_grid.nc').mean(dim='lon')

sst_ERSST_CP = sst_ERSST.sel(lon=slice(180,250)).sst.sel(lat=slice(-5,5)).mean(dim='lat').mean(dim='lon') 
sst_ERSST_CP_DJF = seasonal_data_months(sst_ERSST_CP,[12,1,2])
#sst_ERSST_CP_DJF = sst_ERSST_CP_DJF.sel(time=slice('1950','2018'))

sst_ERSST_EP = sst_ERSST.sel(lon=slice(260,280)).sst.sel(lat=slice(0,10)).mean(dim='lat').mean(dim='lon')
sst_ERSST_EP_DJF = seasonal_data_months(sst_ERSST_EP,[12,1,2])
#sst_ERSST_EP_DJF = sst_ERSST_EP_DJF.sel(time=slice('1950','2018'))

sst_COBE_CP = sst_COBE.sel(lon=slice(180,250)).sst.sel(lat=slice(-5,5)).mean(dim='lat').mean(dim='lon')
sst_COBE_CP_DJF = seasonal_data_months(sst_COBE_CP,[12,1,2])
#sst_COBE_CP_DJF = sst_COBE_CP_DJF.sel(time=slice('1950','2018'))

sst_COBE_EP = sst_COBE.sel(lon=slice(260,280)).sst.sel(lat=slice(0,10)).mean(dim='lat').mean(dim='lon')
sst_COBE_EP_DJF = seasonal_data_months(sst_COBE_EP,[12,1,2])
#sst_COBE_EP_DJF = sst_COBE_EP_DJF.sel(time=slice('1950','2018'))

sst_HadISST_CP = sst_HadISST.sel(lon=slice(180,250)).sst.sel(lat=slice(-5,5)).mean(dim='lat').mean(dim='lon')
sst_HadISST_CP_DJF = seasonal_data_months(sst_HadISST_CP,[12,1,2])
#sst_HadISST_CP_DJF = sst_HadISST_CP_DJF.sel(time=slice('1950','2018'))

sst_HadISST_EP = sst_HadISST.sel(lon=slice(260,280)).sst.sel(lat=slice(0,10)).mean(dim='lat').mean(dim='lon')
sst_HadISST_EP_DJF = seasonal_data_months(sst_HadISST_EP,[12,1,2])
#sst_HadISST_EP_DJF = sst_HadISST_EP_DJF.sel(time=slice('1950','2018'))

sst_Kaplan_CP = sst_Kaplan.sel(lon=slice(180,250)).sst.sel(lat=slice(-5,5)).mean(dim='lat').mean(dim='lon')
sst_Kaplan_CP_DJF = seasonal_data_months(sst_Kaplan_CP,[12,1,2])
#sst_Kaplan_CP_DJF = sst_Kaplan_CP_DJF.sel(time=slice('1950','2018'))

sst_Kaplan_EP = sst_Kaplan.sel(lon=slice(260,280)).sst.sel(lat=slice(0,10)).mean(dim='lat').mean(dim='lon')
sst_Kaplan_EP_DJF = seasonal_data_months(sst_Kaplan_EP,[12,1,2])
#sst_Kaplan_EP_DJF = sst_Kaplan_EP_DJF.sel(time=slice('1950','2018'))

sst_CP_obs = [sst_ERSST_CP_DJF,sst_COBE_CP_DJF,sst_HadISST_CP_DJF,sst_Kaplan_CP_DJF]
sst_EP_obs = [sst_ERSST_EP_DJF,sst_COBE_EP_DJF,sst_HadISST_EP_DJF,sst_Kaplan_EP_DJF]

In [ ]:
### SST data
sst_ERSST_asym = xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mnmean_ERSST_2022_KAPLAN_grid.nc') - xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mnmean_ERSST_2022_KAPLAN_grid.nc').mean(dim='lon')
sst_COBE_asym = xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mon.mean_COBE_2022_KAPLAN_grid.nc') - xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mon.mean_COBE_2022_KAPLAN_grid.nc').mean(dim='lon')
sst_HadISST_asym = xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/HadISST_sst_latest_KAPLAN_grid.nc') - xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/HadISST_sst_latest_KAPLAN_grid.nc').mean(dim='lon')
sst_Kaplan_asym = xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mean.anom_Kaplan_2022_KAPLAN_grid.nc') - xr.open_dataset('/home/jmindlin/causal_EDJ/SST_data/sst.mean.anom_Kaplan_2022_KAPLAN_grid.nc').mean(dim='lon')

sst_ERSST_CP_asym = sst_ERSST_asym.sel(lon=slice(180,250)).sst.sel(lat=slice(-5,5)).mean(dim='lat').mean(dim='lon') 
sst_ERSST_CP_DJF_asym = seasonal_data_months(sst_ERSST_CP_asym,[12,1,2])
#sst_ERSST_CP_DJF = sst_ERSST_CP_DJF.sel(time=slice('1950','2018'))

sst_ERSST_EP_asym = sst_ERSST_asym.sel(lon=slice(260,280)).sst.sel(lat=slice(0,10)).mean(dim='lat').mean(dim='lon')
sst_ERSST_EP_DJF_asym = seasonal_data_months(sst_ERSST_EP_asym,[12,1,2])
#sst_ERSST_EP_DJF = sst_ERSST_EP_DJF.sel(time=slice('1950','2018'))

sst_COBE_CP_asym = sst_COBE_asym.sel(lon=slice(180,250)).sst.sel(lat=slice(-5,5)).mean(dim='lat').mean(dim='lon')
sst_COBE_CP_DJF_asym = seasonal_data_months(sst_COBE_CP_asym,[12,1,2])
#sst_COBE_CP_DJF = sst_COBE_CP_DJF.sel(time=slice('1950','2018'))

sst_COBE_EP_asym = sst_COBE_asym.sel(lon=slice(260,280)).sst.sel(lat=slice(0,10)).mean(dim='lat').mean(dim='lon')
sst_COBE_EP_DJF_asym = seasonal_data_months(sst_COBE_EP_asym,[12,1,2])
#sst_COBE_EP_DJF = sst_COBE_EP_DJF.sel(time=slice('1950','2018'))

sst_HadISST_CP_asym = sst_HadISST_asym.sel(lon=slice(180,250)).sst.sel(lat=slice(-5,5)).mean(dim='lat').mean(dim='lon')
sst_HadISST_CP_DJF_asym = seasonal_data_months(sst_HadISST_CP_asym,[12,1,2])
#sst_HadISST_CP_DJF = sst_HadISST_CP_DJF.sel(time=slice('1950','2018'))

sst_HadISST_EP_asym = sst_HadISST_asym.sel(lon=slice(260,280)).sst.sel(lat=slice(0,10)).mean(dim='lat').mean(dim='lon')
sst_HadISST_EP_DJF_asym = seasonal_data_months(sst_HadISST_EP_asym,[12,1,2])
#sst_HadISST_EP_DJF = sst_HadISST_EP_DJF.sel(time=slice('1950','2018'))

sst_Kaplan_CP_asym = sst_Kaplan_asym.sel(lon=slice(180,250)).sst.sel(lat=slice(-5,5)).mean(dim='lat').mean(dim='lon')
sst_Kaplan_CP_DJF_asym = seasonal_data_months(sst_Kaplan_CP_asym,[12,1,2])
#sst_Kaplan_CP_DJF = sst_Kaplan_CP_DJF.sel(time=slice('1950','2018'))

sst_Kaplan_EP_asym = sst_Kaplan_asym.sel(lon=slice(260,280)).sst.sel(lat=slice(0,10)).mean(dim='lat').mean(dim='lon')
sst_Kaplan_EP_DJF_asym = seasonal_data_months(sst_Kaplan_EP_asym,[12,1,2])
#sst_Kaplan_EP_DJF = sst_Kaplan_EP_DJF.sel(time=slice('1950','2018'))

sst_CP_obs_asym = [sst_ERSST_CP_DJF_asym,sst_COBE_CP_DJF_asym,sst_HadISST_CP_DJF_asym,sst_Kaplan_CP_DJF_asym]
sst_EP_obs_asym = [sst_ERSST_EP_DJF_asym,sst_COBE_EP_DJF_asym,sst_HadISST_EP_DJF_asym,sst_Kaplan_EP_DJF_asym]

In [ ]:
ua_era5_850_djf = seasonal_data(ua_era5_850,'DJF')
ua_ncep_850_djf = seasonal_data(ua_ncep_850,'DJF')
ua_jra55_850_concat_djf = seasonal_data(ua_jra55_850_concat,'DJF')

era5_lat, era5_str = jet_lat_strength(ua_era5_850_djf.u)
ncep_lat, ncep_str = jet_lat_strength(ua_ncep_850_djf.uwnd)
jra55_lat, jra55_str = jet_lat_strength(ua_jra55_850_concat_djf.UGRD_GDS4_HYBL_S123)

era5_lat_zm, era5_str_zm = jet_lat_strength(ua_era5_850_djf.u,0,360)
ncep_lat_zm, ncep_str_zm = jet_lat_strength(ua_ncep_850_djf.uwnd,0,360)
jra55_lat_zm, jra55_str_zm = jet_lat_strength(ua_jra55_850_concat_djf.UGRD_GDS4_HYBL_S123,0,360)

#### Evaluation

##### Pressure Sea Level

In [ ]:
slp = xr.open_dataset('/home/jmindlin/causal_EDJ/NCEP/slp.mon.mean.nc')
slp_DJF = seasonal_data_months(slp,[12,1,2])

In [49]:
# PSL - ERA5
# Initialize class
mlr = spatial_MLR()

# Prepare regression data
variable = standardize_data(detrend_timeseries(slp_DJF.slp.sel(lat=slice(90,-90)).sel(time=slice('1950','2023'))))  # Your target data array
regressors = pd.DataFrame({'spv': standardize_data(detrend_timeseries(stratospheric_polar_vortex_rean[1].uwnd.sel(time=slice('1950','2023')))),
                           'tw': standardize_data(detrend_timeseries(tropical_warming[1].air.sel(time=slice('1950','2023')))),
                           'cp': standardize_data(detrend_timeseries(sst_CP_obs[0].sel(time=slice('1950','2023')))),
                           'ep': standardize_data(detrend_timeseries(sst_EP_obs[0].sel(time=slice('1950','2023'))))})

regressor_names = ['const','spv', 'tw','cp','ep']  # Regressor names
mlr.regression_data(variable, regressors, regressor_names,'era5')

# Perform regression
mlr.perform_regression('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/without_GMST','psl')

# Plot results
regressor_names = ['const','spv', 'tw','cp','ep']  # Regressor names
maps_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/without_GMST/psl/regression_coefficients_era5.nc')
maps_era5_pvals = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/without_GMST/psl/regression_coefficients_pvalues_era5.nc')
maps = [maps_era5[rd] for rd in regressor_names]
maps_pvalues = [maps_era5_pvals[rd] for rd in regressor_names]
titles = ['Climatology','Stratospheric \n Polar Vortex','Tropial Upper Tropospheric \n Temperature','Central Pacific SST','Eastern Pacific SST','Global Mean \n Surface Temperature']
fig = plot_regression_coef_map_MEM(maps, maps_pvalues, titles, '/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/without_GMST/psl/ERA5_detrend_without_GMST_psl_coef.png')

This is regressor_coefs: <xarray.Dataset>
Dimensions:  (lat: 73, lon: 144)
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    const    (lat, lon) float64 -7.133e-09 -7.133e-09 ... -7.3e-09 -7.3e-09
    spv      (lat, lon) float64 0.07049 0.07049 0.07049 ... -0.03163 -0.03163
    tw       (lat, lon) float64 0.2715 0.2715 0.2715 ... -0.09167 -0.09167
    cp       (lat, lon) float64 -0.1012 -0.1012 -0.1012 ... 0.2586 0.2586 0.2586
    ep       (lat, lon) float64 -0.0301 -0.0301 -0.0301 ... -0.1052 -0.1052


<Figure size 2400x1500 with 0 Axes>

In [50]:
# PSL - ERA5
# Initialize class
mlr = spatial_MLR()

# Prepare regression data
variable = standardize_data(detrend_timeseries(slp_DJF.slp.sel(lat=slice(90,-90)).sel(time=slice('1950','2023'))))  # Your target data array
regressors = pd.DataFrame({'spv': standardize_data(detrend_timeseries(stratospheric_polar_vortex_rean[1].uwnd.sel(time=slice('1950','2023')))),
                           'tw': standardize_data(detrend_timeseries(tropical_warming[1].air.sel(time=slice('1950','2023')))),
                           'cp': standardize_data(detrend_timeseries(sst_CP_obs[0].sel(time=slice('1950','2023')))),
                           'ep': standardize_data(detrend_timeseries(sst_EP_obs[0].sel(time=slice('1950','2023')))),
                           'gmst': standardize_data(detrend_timeseries(tas_DJF.sel(time=slice('1950','2023'))))})

regressor_names = ['const','spv', 'tw','cp','ep','gmst']  # Regressor names
mlr.regression_data(variable, regressors, regressor_names,'era5')

# Perform regression
mlr.perform_regression('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST','psl')

# Plot results
regressor_names = ['const','spv', 'tw','cp','ep','gmst']  # Regressor names
maps_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/psl/regression_coefficients_era5.nc')
maps_era5_pvals = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/psl/regression_coefficients_pvalues_era5.nc')
maps = [maps_era5[rd] for rd in regressor_names]
maps_pvalues = [maps_era5_pvals[rd] for rd in regressor_names]
titles = ['Climatology','Stratospheric \n Polar Vortex','Tropial Upper Tropospheric \n Temperature','Central Pacific SST','Eastern Pacific SST','Global Mean \n Surface Temperature']
fig = plot_regression_coef_map_MEM(maps, maps_pvalues, titles, '/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/psl/ERA5_detrend_with_GMST_psl_coef.png')

This is regressor_coefs: <xarray.Dataset>
Dimensions:  (lat: 73, lon: 144)
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    const    (lat, lon) float64 -1.052e-08 -1.052e-08 ... -7.944e-09 -7.944e-09
    spv      (lat, lon) float64 0.1087 0.1087 0.1087 ... -0.02437 -0.02437
    tw       (lat, lon) float64 0.5696 0.5696 0.5696 ... -0.03506 -0.03506
    cp       (lat, lon) float64 -0.1611 -0.1611 -0.1611 ... 0.2473 0.2473 0.2473
    ep       (lat, lon) float64 -0.0008328 -0.0008328 ... -0.0996 -0.0996
    gmst     (lat, lon) float64 -0.3527 -0.3527 -0.3527 ... -0.06697 -0.06697


<Figure size 2400x1500 with 0 Axes>

In [51]:
# PSL - ERA5
# Initialize class
mlr = spatial_MLR()

# Prepare regression data
variable = standardize_data(slp_DJF.sel(lat=slice(90,-90)).sel(time=slice('1950','2023')).slp)  # Your target data array
regressors = pd.DataFrame({'spv': standardize_data(stratospheric_polar_vortex_rean[1].uwnd.sel(time=slice('1950','2023'))),
                           'tw': standardize_data(tropical_warming[1].air.sel(time=slice('1950','2023'))),
                           'cp': standardize_data(sst_CP_obs[0].sel(time=slice('1950','2023'))),
                           'ep': standardize_data(sst_EP_obs[0].sel(time=slice('1950','2023'))),
                           'gmst': standardize_data(tas_DJF.sel(time=slice('1950','2023')))})

regressor_names = ['const','spv', 'tw','cp','ep','gmst']  # Regressor names
mlr.regression_data(variable, regressors, regressor_names,'era5')

# Perform regression
mlr.perform_regression('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/with_GMST','psl')

# Plot results
regressor_names = ['const','spv', 'tw','cp','ep','gmst']  # Regressor names
maps_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/with_GMST/psl/regression_coefficients_era5.nc')
maps_era5_pvals = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/with_GMST/psl/regression_coefficients_pvalues_era5.nc')
maps = [maps_era5[rd] for rd in regressor_names]
maps_pvalues = [maps_era5_pvals[rd] for rd in regressor_names]
titles = ['Climatology','Stratospheric \n Polar Vortex','Tropical Upper Tropospheric \n Temperature','Central Pacific SST','Eastern Pacific SST','Global Mean \n Surface Temperature']
fig = plot_regression_coef_map_MEM(maps, maps_pvalues, titles, '/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/with_GMST/psl/ERA5_trend_with_GMST_psl_coef.png')

This is regressor_coefs: <xarray.Dataset>
Dimensions:  (lat: 73, lon: 144)
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    const    (lat, lon) float64 -3.098e-05 -3.098e-05 ... -6.246e-05 -6.246e-05
    spv      (lat, lon) float64 0.06934 0.06934 0.06934 ... -0.004681 -0.004681
    tw       (lat, lon) float64 0.4533 0.4533 0.4533 ... 0.03807 0.03807 0.03807
    cp       (lat, lon) float64 -0.2132 -0.2132 -0.2132 ... 0.2576 0.2576 0.2576
    ep       (lat, lon) float64 0.05377 0.05377 0.05377 ... -0.1247 -0.1247
    gmst     (lat, lon) float64 -0.3053 -0.3053 -0.3053 ... -0.3594 -0.3594


<Figure size 2400x1500 with 0 Axes>

In [52]:
# PSL - ERA5
# Initialize class
mlr = spatial_MLR()

# Prepare regression data
variable = standardize_data(slp_DJF.sel(lat=slice(90,-90)).sel(time=slice('1950','2023')).slp)  # Your target data array
regressors = pd.DataFrame({'spv': standardize_data(stratospheric_polar_vortex_rean[1].uwnd.sel(time=slice('1950','2023'))),
                           'tw': standardize_data(tropical_warming[1].air.sel(time=slice('1950','2023'))),
                           'cp': standardize_data(sst_CP_obs[0].sel(time=slice('1950','2023'))),
                           'ep': standardize_data(sst_EP_obs[0].sel(time=slice('1950','2023')))})

regressor_names = ['const','spv', 'tw','cp','ep']  # Regressor names
mlr.regression_data(variable, regressors, regressor_names,'era5')

# Perform regression
mlr.perform_regression('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/without_GMST','psl')

# Plot results
regressor_names = ['const','spv', 'tw','cp','ep']  # Regressor names
maps_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/without_GMST/psl/regression_coefficients_era5.nc')
maps_era5_pvals = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/without_GMST/psl/regression_coefficients_pvalues_era5.nc')
maps = [maps_era5[rd] for rd in regressor_names]
maps_pvalues = [maps_era5_pvals[rd] for rd in regressor_names]
titles = ['Climatology','Stratospheric \n Polar Vortex','Tropical Upper Tropospheric \n Temperature','Central Pacific SST','Eastern Pacific SST','Global Mean \n Surface Temperature']
fig = plot_regression_coef_map_MEM(maps, maps_pvalues, titles, '/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/without_GMST/psl/ERA5_trend_without_GMST_psl_coef.png')

This is regressor_coefs: <xarray.Dataset>
Dimensions:  (lat: 73, lon: 144)
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    const    (lat, lon) float64 -2.543e-05 -2.543e-05 ... -5.592e-05 -5.592e-05
    spv      (lat, lon) float64 0.1018 0.1018 0.1018 ... 0.03352 0.03352 0.03352
    tw       (lat, lon) float64 0.2202 0.2202 0.2202 ... -0.2363 -0.2363 -0.2363
    cp       (lat, lon) float64 0.07038 0.07038 0.07038 ... 0.5915 0.5915 0.5915
    ep       (lat, lon) float64 -0.1754 -0.1754 -0.1754 ... -0.3945 -0.3945


<Figure size 2400x1500 with 0 Axes>

##### Westerly winds

In [ ]:
# U850 - ERA5
# Initialize class
mlr = spatial_MLR()

# Prepare regression data
variable = standardize_data(ua_era5_850_djf.sel(lat=slice(0,-90)).sel(time=slice('1950','2023')).u)  # Your target data array
regressors = pd.DataFrame({'spv': standardize_data(stratospheric_polar_vortex_rean[0].sel(time=slice('1950','2023'))),
                           'tw': standardize_data(tropical_warming[0].t.sel(time=slice('1950','2023'))),
                           'cp': standardize_data(sst_CP_obs[0].sel(time=slice('1950','2023'))),
                           'ep': standardize_data(sst_EP_obs[0].sel(time=slice('1950','2023')))})

regressor_names = ['const','spv', 'tw','cp','ep']  # Regressor names
mlr.regression_data(variable, regressors, regressor_names,'era5')

# Perform regression
mlr.perform_regression('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/with_GMST','u850')

In [51]:
# ERA5 - U850
# Initialize class
mlr = spatial_MLR()

# Prepare regression data
variable = standardize_data(detrend_timeseries(ua_era5_850_djf.sel(lat=slice(0,-90)).sel(time=slice('1950','2023')).u))  # Your target data array
regressors = pd.DataFrame({'spv': standardize_data(detrend_timeseries(stratospheric_polar_vortex_rean[0].sel(time=slice('1950','2023')))), 
                           'tw': standardize_data(detrend_timeseries(tropical_warming[0].t.sel(time=slice('1950','2023')))), 
                           'cp': standardize_data(detrend_timeseries(sst_CP_obs[0].sel(time=slice('1950','2023')))), 
                           'ep': standardize_data(detrend_timeseries(sst_EP_obs[0].sel(time=slice('1950','2023')))),
                           'gmst': standardize_data(detrend_timeseries(tas_DJF.sel(time=slice('1950','2023')))),})

regressor_names = ['const','spv', 'tw','cp','ep','gmst']  # Regressor names
mlr.regression_data(variable, regressors, regressor_names,'era5')

# Perform regression
mlr.perform_regression('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST','u850')

In [50]:
# U850 - ERA5
# Initialize class
mlr = spatial_MLR()

# Prepare regression data
variable = standardize_data(detrend_timeseries(ua_era5_850_djf.sel(lat=slice(0,-90)).sel(time=slice('1950','2023')).u))  # Your target data array
regressors = pd.DataFrame({'spv': standardize_data(detrend_timeseries(stratospheric_polar_vortex_rean[0].sel(time=slice('1950','2023')))),
                           'tw': standardize_data(detrend_timeseries(tropical_warming[0].t.sel(time=slice('1950','2023')))),
                           'cp': standardize_data(detrend_timeseries(sst_CP_obs[0].sel(time=slice('1950','2023')))),
                           'ep': standardize_data(detrend_timeseries(sst_EP_obs[0].sel(time=slice('1950','2023'))))})

regressor_names = ['const','spv', 'tw','cp','ep']  # Regressor names
mlr.regression_data(variable, regressors, regressor_names,'era5')

# Perform regression
mlr.perform_regression('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/without_GMST','u850')



This is regressor_coefs: <xarray.Dataset>
Dimensions:  (lon: 1440, lat: 361)
Coordinates:
  * lon      (lon) float32 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * lat      (lat) float32 0.0 -0.25 -0.5 -0.75 ... -89.25 -89.5 -89.75 -90.0
    level    int32 850
Data variables:
    const    (lat, lon) float64 3.005e-08 9.852e-09 ... 8.685e-09 8.685e-09
    spv      (lat, lon) float64 -0.05366 -0.05166 -0.04958 ... 0.07634 0.07634
    tw       (lat, lon) float64 -0.387 -0.3898 -0.3934 ... -0.3068 -0.3068
    cp       (lat, lon) float64 1.505 1.504 1.503 1.503 ... 0.2285 0.2285 0.2285
    ep       (lat, lon) float64 -0.5268 -0.5206 -0.5144 ... 0.0461 0.0461 0.0461


In [46]:
# Initialize class
mlr = spatial_MLR()

# Prepare regression data
variable = standardize_data(detrend_timeseries(ua_jra55_850_concat_djf.UGRD_GDS4_HYBL_S123.sel(lat=slice(0,-90)).sel(time=slice('1958','2023'))))  # Your target data array
regressors = pd.DataFrame({'spv': standardize_data(detrend_timeseries(stratospheric_polar_vortex_rean[2].UGRD_GDS4_HYBL_S123.sel(time=slice('1958','2023')))), 
                           'tw': standardize_data(detrend_timeseries(tropical_warming[2].TMP_GDS4_HYBL_S123.sel(time=slice('1958','2023')))), 
                           'cp': standardize_data(detrend_timeseries(sst_CP_obs[0].sel(time=slice('1958','2023')))), 
                           'ep': standardize_data(detrend_timeseries(sst_EP_obs[0].sel(time=slice('1958','2023')))),
                           'gmst': standardize_data(detrend_timeseries(tas_DJF.sel(time=slice('1958','2023')))),})

regressor_names = ['const','spv', 'tw','cp','ep','gmst']  # Regressor names
mlr.regression_data(variable, regressors, regressor_names,'JRA55')

# Perform regression
mlr.perform_regression('/home/jmindlin/causal_EDJ/JRA55_causal_network/anomalies/detrended_data/with_GMST/','u850')

This is regressor_coefs: <xarray.Dataset>
Dimensions:   (lon: 640, lat: 160, lv_HYBL1: 1)
Coordinates:
  * lon       (lon) float32 0.0 0.5625 1.125 1.688 ... 357.8 358.3 358.9 359.4
  * lat       (lat) float32 -0.2808 -0.8424 -1.404 ... -88.45 -89.01 -89.57
  * lv_HYBL1  (lv_HYBL1) int32 12
Data variables:
    const     (lv_HYBL1, lat, lon) float64 -4.816e-10 2.069e-08 ... -2.95e-10
    spv       (lv_HYBL1, lat, lon) float64 -0.2602 -0.2622 ... 0.2657 0.2617
    tw        (lv_HYBL1, lat, lon) float64 -0.2484 -0.2413 ... -0.2206 -0.2161
    cp        (lv_HYBL1, lat, lon) float64 0.5045 0.5255 ... -0.1084 -0.1172
    ep        (lv_HYBL1, lat, lon) float64 -0.5382 -0.544 ... 0.1362 0.1357
    gmst      (lv_HYBL1, lat, lon) float64 0.00535 0.001355 ... 0.1795 0.1818


In [38]:
# Initialize class
mlr = spatial_MLR()

# Prepare regression data
variable = standardize_data(detrend_timeseries(ua_ncep_850_djf.sel(lat=slice(0,-90)).sel(time=slice('1950','2023')).uwnd))  # Your target data array
regressors = pd.DataFrame({'spv': standardize_data(detrend_timeseries(stratospheric_polar_vortex_rean[1].uwnd.sel(time=slice('1950','2023')))), 
                           'tw': standardize_data(detrend_timeseries(tropical_warming[1].air.sel(time=slice('1950','2023')))), 
                           'cp': standardize_data(detrend_timeseries(sst_CP_obs[0].sel(time=slice('1950','2023')))), 
                           'ep': standardize_data(detrend_timeseries(sst_EP_obs[0].sel(time=slice('1950','2023')))),
                           'gmst': standardize_data(detrend_timeseries(tas_DJF.sel(time=slice('1950','2023')))),})

regressor_names = ['const','spv', 'tw','cp','ep','gmst']  # Regressor names
mlr.regression_data(variable, regressors, regressor_names,'NCEP')

# Perform regression
mlr.perform_regression('/home/jmindlin/causal_EDJ/NCEP_causal_network/anomalies/detrended_data/with_GMST/','u850')

This is regressor_coefs: <xarray.Dataset>
Dimensions:  (lat: 37, lon: 144)
Coordinates:
    level    float32 850.0
  * lat      (lat) float32 0.0 -2.5 -5.0 -7.5 -10.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    const    (lat, lon) float64 6.751e-09 3.203e-08 ... -3.72e-09 -6.378e-09
    spv      (lat, lon) float64 -0.02017 -0.02882 -0.05748 ... 0.04026 0.03192
    tw       (lat, lon) float64 -0.3206 -0.3706 -0.4323 ... -0.489 -0.509
    cp       (lat, lon) float64 -0.5302 -0.4383 -0.3174 ... -0.1717 -0.1509
    ep       (lat, lon) float64 0.6051 0.6061 0.6051 ... 0.5148 0.5131 0.5136
    gmst     (lat, lon) float64 0.149 0.07994 -0.02344 ... 0.1369 0.1354 0.1313


In [12]:
# Initialize class
mlr = spatial_MLR()

# Prepare regression data
variable = standardize_data(detrend_timeseries(ua_era5_850_djf.sel(lat=slice(0,-90)).sel(time=slice('1950','2023')).u))  # Your target data array
regressors = pd.DataFrame({'spv': standardize_data(detrend_timeseries(stratospheric_polar_vortex_rean[0].sel(time=slice('1950','2023')))), 
                           'tw': standardize_data(detrend_timeseries(tropical_warming[0].t.sel(time=slice('1950','2023')))), 
                           'cp': standardize_data(detrend_timeseries(sst_CP_obs[0].sel(time=slice('1950','2023')))), 
                           'ep': standardize_data(detrend_timeseries(sst_EP_obs[0].sel(time=slice('1950','2023')))),
                           'gmst': standardize_data(detrend_timeseries(tas_DJF.sel(time=slice('1950','2023')))),})

regressor_names = ['const','spv', 'tw','cp','ep','gmst']  # Regressor names
mlr.regression_data(variable, regressors, regressor_names,'era5')

# Perform regression
mlr.perform_regression('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/','u850')

This is regressor_coefs: <xarray.Dataset>
Dimensions:  (lon: 1440, lat: 361)
Coordinates:
  * lon      (lon) float32 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * lat      (lat) float32 0.0 -0.25 -0.5 -0.75 ... -89.25 -89.5 -89.75 -90.0
    level    int32 850
Data variables:
    const    (lat, lon) float64 3.092e-08 1.069e-08 ... 7.131e-09 7.131e-09
    spv      (lat, lon) float64 -0.0433 -0.04162 -0.03992 ... 0.05781 0.05781
    tw       (lat, lon) float64 -0.3116 -0.3167 -0.3231 ... -0.4418 -0.4418
    cp       (lat, lon) float64 1.476 1.475 1.475 1.476 ... 0.282 0.282 0.282
    ep       (lat, lon) float64 -0.4963 -0.491 -0.4859 ... -0.008573 -0.008573
    gmst     (lat, lon) float64 -0.1098 -0.1064 -0.1024 ... 0.1964 0.1964 0.1964


In [20]:
regressor_names = ['const','gmst','spv','tw','cp','ep']  # Regressor names
maps_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/with_GMST/u850/regression_coefficients_era5.nc')
maps_era5_pvals = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/with_GMST/u850/regression_coefficients_pvalues_era5.nc')
R2_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/anomalies/detrended_data/with_GMST/u850/R2_era5.nc')
maps = [maps_era5[rd] for rd in regressor_names]
maps_pvalues = [maps_era5_pvals[rd] for rd in regressor_names]
maps[0] = R2_era5.__xarray_dataarray_variable__ * 100
titles = ['Fraction of \n Variance Explained','Global Mean \n Surface Temperature','Stratospheric \n Polar Vortex','Tropical Upper Tropospheric \n Temperature','Central Pacific SST','Eastern Pacific SST']
fig = plot_regression_coef_map_MEM(maps, maps_pvalues, titles, '/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/trended_data/with_GMST/u850/ERA5_trend_with_GMST_ua850_coef.png')

<Figure size 2400x1500 with 0 Axes>

In [48]:
regressor_names = ['const','gmst','spv','tw','cp','ep']  # Regressor names
maps_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/anomalies/detrended_data/with_GMST/u850/regression_coefficients_era5.nc')
maps_era5_pvals = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/anomalies/detrended_data/with_GMST/u850/regression_coefficients_pvalues_era5.nc')
R2_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/anomalies/detrended_data/with_GMST/u850/R2_era5.nc')
maps = [maps_era5[rd] for rd in regressor_names]
maps_pvalues = [maps_era5_pvals[rd] for rd in regressor_names]
maps[0] = R2_era5.__xarray_dataarray_variable__ * 100
titles = ['Fraction of \n Variance Explained','Global Mean \n Surface Temperature','Stratospheric \n Polar Vortex','Tropical Upper Tropospheric \n Temperature','Central Pacific SST','Eastern Pacific SST']
fig = plot_regression_coef_map_row(maps, maps_pvalues, titles, '/home/jmindlin/causal_EDJ/ERA5_causal_network/anomalies/detrended_data/with_GMST/u850/ERA5_detrend_with_GMST_ua850_coef_row.png')

<Figure size 3000x1000 with 0 Axes>

In [23]:
regressor_names = ['const','gmst','spv','tw','cp','ep']  # Regressor names
maps_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/u850/regression_coefficients_era5.nc')
maps_era5_pvals = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/u850/regression_coefficients_pvalues_era5.nc')
R2_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/u850/R2_era5.nc')
maps = [maps_era5[rd] for rd in regressor_names]
maps_pvalues = [maps_era5_pvals[rd] for rd in regressor_names]
maps[0] = R2_era5.__xarray_dataarray_variable__ * 100
titles = ['Fraction of \n Variance Explained','Global Mean \n Surface Temperature','Stratospheric \n Polar Vortex','Tropical Upper Tropospheric \n Temperature','Central Pacific SST','Eastern Pacific SST']
fig = plot_regression_coef_map_MEM(maps, maps_pvalues, titles, '/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/u850/ERA5_detrend_with_GMST_ua850_coef.png')

<Figure size 2400x1500 with 0 Axes>

In [1]:
regressor_names = ['const','gmst','spv','tw','cp','ep']  # Regressor names
maps_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/u850/regression_coefficients_era5.nc')
maps_era5_pvals = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/u850/regression_coefficients_pvalues_era5.nc')
R2_era5 = xr.open_dataset('/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/u850/R2_era5.nc')
maps = [maps_era5[rd] for rd in regressor_names]
maps_pvalues = [maps_era5_pvals[rd] for rd in regressor_names]
maps[0] = R2_era5.__xarray_dataarray_variable__ * 100
titles = ['Fraction of \n Variance Explained','Global Mean \n Surface Temperature','Stratospheric \n Polar Vortex','Tropical Upper Tropospheric \n Temperature','Central Pacific SST','Eastern Pacific SST']
fig = plot_regression_coef_map_row(maps, maps_pvalues, titles, '/home/jmindlin/causal_EDJ/ERA5_causal_network/raw_data/detrended_data/with_GMST/u850/ERA5_detrend_with_GMST_ua850_coef_row.png')